### Install liten
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ liten
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten/dist/liten-0.0.1-py3-none-any.whl
```

Import Apache arrow

In [1]:
import pyarrow as pa
from pyarrow import csv

Import Liten-ten is local rten is remote. rten imports pyarrow library as well.

In [2]:
import liten as ten

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten._liten' does not match runtime version 3.8
<frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [3]:
import liten.rcliten as rten

Import Ray to be used as a cluster

In [4]:
import ray

Start a cluster with single worker.

In [5]:
ray.init(num_cpus=1)

2021-08-10 20:53:34,143	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.74.253',
 'raylet_ip_address': '192.168.74.253',
 'redis_address': '192.168.74.253:6379',
 'object_store_address': '/tmp/ray/session_2021-08-10_20-53-33_406807_2349/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-10_20-53-33_406807_2349/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-10_20-53-33_406807_2349',
 'metrics_export_port': 65282,
 'node_id': 'cdec165098e022ba1746192fcb153ab2247558e2'}

In [6]:
ray.cluster_resources()

{'object_store_memory': 21.0,
 'node:192.168.74.253': 1.0,
 'memory': 63.0,
 'CPU': 1.0}

Create a Liten Cache Actor. It is residing on a remote node, and being executed on that node. tc is the Liten Cache actor handle.

In [7]:
rten.RCLiten = ray.remote(rten.RCLiten)
tc = rten.RCLiten.remote()

(pid=2434) <frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten._liten' does not match runtime version 3.8
(pid=2434) <frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject
(pid=2434) WARNING: Logging before InitGoogleLogging() is written to STDERR
(pid=2434) I20210810 20:54:11.729995  2434 TCache.cpp:24] Created a new TCache


These are fact and dimension tables of TPCH. Read them remotely.

In [8]:
fact_tables = ['lineitem']
dim_tables = ['customer','orders','supplier','nation','region']
tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/'

In [9]:
def read_tables(tables, table_type):
    arrow_tables = []
    for table_name in tables:
        tpch_table = tpch_dir+table_name+'.tbl'
        print('Reading ', tpch_table)
        tc.set_table.remote(table_name, table_type)
        pytable = tc.read_csv.remote(input_file=tpch_table, parse_options=csv_options)
        # print(' Rows=', pytable.num_rows,' Cols=', pytable.num_columns)
        arrow_tables.append(pytable)
    return arrow_tables

In [10]:
%%time
csv_options = pa.csv.ParseOptions(delimiter='|')
pa_fact_tables = read_tables(fact_tables, 1)
pa_dim_tables = read_tables(dim_tables, 0)

Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/lineitem.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/customer.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/orders.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/supplier.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/nation.tbl
Reading  /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/region.tbl
CPU times: user 22.3 ms, sys: 22.9 ms, total: 45.2 ms
Wall time: 26 ms


(pid=2434) I20210810 20:54:53.529749  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:53.530014  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:53.533051  2434 TCatalog.cpp:17] Created a new TCatalog


(pid=2434) Added Table= b'lineitem'


(pid=2434) I20210810 20:54:53.838146  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:53.838274  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:53.838488  2434 TCatalog.cpp:17] Created a new TCatalog


(pid=2434) Added Table= b'customer'


(pid=2434) I20210810 20:54:57.007773  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.007866  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.008538  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.092166  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.092278  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.092495  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.126919  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.127034  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.127377  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.165297  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.165406  2434 TCatalog.cpp:17] Created a new TCatalog
(pid=2434) I20210810 20:54:57.165578  2434 TCatalog.cpp:17] Created a new TCatalog


(pid=2434) Added Table= b'orders'
(pid=2434) Added Table= b'supplier'
(pid=2434) Added Table= b'nation'
(pid=2434) Added Table= b'region'


In [11]:
tc.info.remote()

ObjectRef(fafba2bafaed5dc3df5a1a820100000001000000)

(pid=2434) I20210810 20:55:12.035115  2434 TConfigs.cpp:16] Created a new TConfigs
(pid=2434) I20210810 20:55:12.035182  2434 TCatalog.cpp:17] Created a new TCatalog


Read a table into TCache

In [12]:
%%time
result = tc.make_dtensor.remote()

CPU times: user 0 ns, sys: 1.86 ms, total: 1.86 ms
Wall time: 1.23 ms


Read Arrow table

In [13]:
result = tc.query6.remote()

(pid=2434)  TPCH QUERY 6 
(pid=2434) SELECT 
(pid=2434)   SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE 
(pid=2434) FROM 
(pid=2434)   LINEITEM
(pid=2434) WHERE
(pid=2434)   L_SHIPDATE >= DATE '1997-01-01'
(pid=2434)   AND L_SHIPDATE < DATE '1997-01-01' + INTERVAL '1' YEAR
(pid=2434)   AND L_DISCOUNT BETWEEN 0.07 - 0.01 AND 0.07 + 0.01
(pid=2434)   AND L_QUANTITY < 25;
(pid=2434) 
(pid=2434) Revenue= 156594095.60960016
(pid=2434) 


In [14]:
result = tc.query5.remote()

(pid=2434)  
(pid=2434) SELECT
(pid=2434) 	N_NAME,
(pid=2434) 	SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE
(pid=2434) FROM
(pid=2434) 	CUSTOMER,
(pid=2434) 	ORDERS,
(pid=2434) 	LINEITEM,
(pid=2434) 	SUPPLIER,
(pid=2434) 	NATION,
(pid=2434) 	REGION
(pid=2434) WHERE
(pid=2434) 	C_CUSTKEY = O_CUSTKEY
(pid=2434) 	AND L_ORDERKEY = O_ORDERKEY
(pid=2434) 	AND L_SUPPKEY = S_SUPPKEY
(pid=2434) 	AND C_NATIONKEY = S_NATIONKEY
(pid=2434) 	AND S_NATIONKEY = N_NATIONKEY
(pid=2434) 	AND N_REGIONKEY = R_REGIONKEY
(pid=2434) 	AND R_NAME = 'EUROPE'
(pid=2434) 	AND O_ORDERDATE >= DATE '1995-01-01'
(pid=2434) 	AND O_ORDERDATE < DATE '1995-01-01' + INTERVAL '1' YEAR
(pid=2434) GROUP BY
(pid=2434) 	N_NAME
(pid=2434) ORDER BY
(pid=2434) 	REVENUE DESC;
(pid=2434) 
(pid=2434) b'RUSSIA' = 32382.172400000003
(pid=2434) b'FRANCE' = 45906.1421
(pid=2434) b'GERMANY' = 101655.74960000001
(pid=2434) 


This will kill remote Liten Cache.

In [ ]:
ray.kill(tc)

Shut down ray now

In [ ]:
ray.shutdown()